In [56]:
import os
import sys
import platform
import pandas as pd
import re
from datetime import datetime, timedelta

3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0]


In [57]:
import requests
from bs4 import BeautifulSoup
import html2text
from datetime import datetime

In [1]:
# get pdf download link, meeting date, registration date
# from_date : start date
def get_minutes_list(from_date='20050101'):
    prefix_addr = "https://www.bok.or.kr"
    from_date = datetime.strptime(from_date, '%Y%m%d')

#     page index is from 1 to 30
    for pageIndex in range(1, 31):
        url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(pageIndex)
        user_agent = 'Mozilla/5.0'
        headers ={'User-Agent' : user_agent}
        page = requests.get(url, headers=headers)

        try:
            soup = BeautifulSoup(page.content, 'html.parser')
#             get minutes board list
            brdList = soup.find_all('span', class_='col m10 s10 x9 ctBx')
#             get registration date of minutes
            datainfo = soup.find_all('div', class_='col s12 dataInfo')

            for post, data in zip(brdList, datainfo):
#                 get minutes pdf download link
                guid = prefix_addr+post.a['href'][:-12]

#                 find description
                desPage = requests.get(guid)
                desSoup = BeautifulSoup(desPage.content, 'html.parser')   
                description = desSoup.find('div', class_='dbData').get_text().strip()
                if description.replace(' ','').find('통화정책방향') >= 0:
#                     get title
                    title = post.find('span', class_='titlesub').get_text().strip()
        
#                     get meeting date
                    mdate = title[title.find(')(') + 2:-1]            
                    if mdate[-1] == '.':
                        mdate = mdate[:-1]
                    mdate = datetime.strptime(mdate, '%Y.%m.%d')

#                     get data until from_date
                    if mdate < from_date:
                        break

#                     get registration date
                    rdate = data.find('span', class_='date').get_text().strip()
                    rdate = datetime.strptime(rdate[3:], '%Y.%m.%d')    

                    get_minutes_file(guid, mdate, rdate)
        except:
            print("get url.content error and pass page{} it".format(pageIndex))

In [59]:
# download minutes pdf file
def get_minutes_file(page_addr, mdate, rdate):
    file_header = 'data/minutes/pdf/KO_'
    prefix_addr = "http://bok.or.kr"

    page = requests.get(page_addr)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    try:
        links = soup.find('div', class_='addfile').find_all('a')

        for link in links:
            filename = link.get_text()
            filename = filename.replace('\r', '').replace('\t', '').replace('\n', '')

            if filename[-3:] == 'pdf':
                filename = mdate.strftime('%Y%m%d') + "_" + rdate.strftime('%Y%m%d')+ '.pdf'
                file_addr = prefix_addr + link["href"]
                file_res = requests.get(file_addr)
                filepath = file_header + filename

#                 save file
                with open(filepath, 'wb') as f:
                    f.write(file_res.content)

                print('save file name : ')
                print(filename)
    except:
        print("get file failed and pass it")

In [62]:
# download and save minutes pdf file
get_minutes_list()

print(filename, file_addr)
20190228_20190319.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000009690&fileSn=2
print(filename, file_addr)
20190124_20190212.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000009054&fileSn=2
print(filename, file_addr)
20181130_20181218.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000007974&fileSn=1
print(filename, file_addr)
20181018_20181106.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000007208&fileSn=1
print(filename, file_addr)
20180831_20180918.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000006297&fileSn=2
print(filename, file_addr)
20180712_20180731.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000005384&fileSn=2
print(filename, file_addr)
20180524_20180612.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuN

print(filename, file_addr)
20131010_20131029.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000101175&fileSn=2
print(filename, file_addr)
20130912_20131001.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000100570&fileSn=2
print(filename, file_addr)
20130808_20130827.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000099785&fileSn=2
print(filename, file_addr)
20130711_20130730.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000099057&fileSn=2
print(filename, file_addr)
20130613_20130702.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000098432&fileSn=2
print(filename, file_addr)
20130509_20130528.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000097596&fileSn=2
print(filename, file_addr)
20130411_20130430.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuN

print(filename, file_addr)
20090312_20090428.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000085936&fileSn=2
print(filename, file_addr)
20090212_20090331.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000085938&fileSn=2
print(filename, file_addr)
20090109_20090224.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000085920&fileSn=2
print(filename, file_addr)
20081211_20090128.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000085923&fileSn=2
print(filename, file_addr)
20081107_20081223.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000085925&fileSn=2
print(filename, file_addr)
20081027_20081209.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000085926&fileSn=2
print(filename, file_addr)
20081009_20081125.pdf http://bok.or.kr/portal/cmmn/file/fileDown.do?menuN